In [1]:
import pandas as pd
import numpy as np
import re
import spacy
from spacy.lang.en import English
from spacy import displacy
nlp = spacy.load('en_core_web_sm', disable=["ner", "textcat", "entity_ruler", "merge_noun_chunks", "merge_entities", "merge_subtokens"])

In [2]:
from IPython.display import Image
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import seaborn as sns
import matplotlib.pylab as plt
from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
from xgboost import XGBClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import KFold
from nltk.corpus import stopwords
import string

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.linear_model import SGDClassifier, LogisticRegression,LogisticRegressionCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC,LinearSVC
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors

In [ ]:
options = {"compact": True, "bg": "#09a3d5",
           "color": "white", "font": "Source Sans Pro","collapse_phrases":False}

In [ ]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('max_colwidth', 260)

In [ ]:
dfExoplanetsNASAannot = pd.read_json('../data/dfExoplanetsNASAdetected100rand_v2.json', orient = 'table')
del dfExoplanetsNASAannot['tagRootSent']
del dfExoplanetsNASAannot['tagDetected']
dfExoplanetsNASAannot.head(20)

In [ ]:
def satz_analytic2(satz):
#    merkmal = dict({"satz":satz.text})
    merkmal=dict({})
# search for main verb
    for t in satz:
        if t.dep_ == "ROOT":
            r=t.head.text
            merkmal.update({"act":r})
# subject and object related to verb
    for t in satz:
#        if t.dep_ == "dobj" and merkmal["act"]==t.head.text:
#            merkmal.update({"obj":''.join(w.text_with_ws for w in t.subtree)})
        if t.dep_ == "nsubj" and merkmal["act"]==t.head.text:
            merkmal.update({"subject":t.text.lower()})

    return(merkmal)


In [ ]:
dfExoplanetsNASAannot["s"]=dfExoplanetsNASAannot["sent"].apply(lambda y: satz_analytic2(nlp(y)))
dfExoplanetsNASAannot.head(20)

In [ ]:
_all_xs=list(dfExoplanetsNASAannot["sent"])


In [ ]:
_all_ys=np.array(list(dfExoplanetsNASAannot["label"]))

In [ ]:
def transp(x):
    if x=="discovery":
        y=1
    else:
        y=0
    return(y)
dfExoplanetsNASAannot["label"]=dfExoplanetsNASAannot["label"].apply(lambda x: transp(x))

In [ ]:
dfExoplanetsNASAannot.head(10)

In [ ]:
_all_xs[1]

In [ ]:
X=list(dfExoplanetsNASAannot["sent"])
y=np.array(list(dfExoplanetsNASAannot["label"]))



In [ ]:
from collections import Counter
Category=Counter(y)
Counter=dict(Category)

In [ ]:
Counter

In [ ]:
plt.bar(list(Counter.keys()),Counter.values(),color="r",width=.4,tick_label=["1:discovery","0:None"])
plt.title("Distribution of samples over different classes")


In [ ]:
X,y = shuffle(X, y, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(
X,y, test_size=0.25, random_state=42)    


In [ ]:
len(y)

# Baseline models

while training and buliding a model, we should keep in mind that there is never the best one. It is realtively experimental process. We start to try different trials. we trired different evaluation metrics

We have tried the following models:
- Multinomial Naïve Bayes
- XGBoosts classifier
- Random Forest
- Support Vector Machine
-  SGD Classifier


In [ ]:
def train_predict(clf,X_train,y_train,X_test,y_test):
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    return y_pred

In [ ]:
def cross_validation_accuracy(clf,X,y):
    scores = cross_val_score(clf, X, y, cv=3)
    print(scores)
    print('Accuracy of : {:.3f} ± {:.3f}'.format(np.mean(scores), 2 * np.std(scores)))

In [ ]:
def Color_Confusion_Matrix(y, y_test, y_pred):
    cm=confusion_matrix(y_test, y_pred)
    l=len(set(y))
    df_cm = pd.DataFrame(cm, range(l), range(l))
    sns.set(font_scale=1)
    sns.heatmap(df_cm,cmap="Blues", annot=True,annot_kws={"size": 16})
    plt.ylabel('True label');
    plt.xlabel('Predicted label');
    plt.title("Confusion Matrix", size = 16)
    plt.savefig('CM_Test_01.png')

In [ ]:
from sklearn.metrics import roc_curve  
from sklearn.metrics import roc_auc_score 
def plot_roc_curve(fpr, tpr,auc):  
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve with AUC {:.3f}'.format(AUC))
    plt.legend()
    plt.show()

# Baseline Models

In [ ]:
## 01-Naive Bayes
trial_01=Pipeline([ ('vectorizer', TfidfVectorizer()), ('classifier', MultinomialNB())])
################################################################################

## 02-XGBClassifier
trial_02 = Pipeline([('vect', CountVectorizer(stop_words='english')),
                     ('clf', XGBClassifier())])
################################################################################
## 03-Random Forest Classifier
trial_03= Pipeline([('vect', CountVectorizer(stop_words='english')),
                     ('clf', RandomForestClassifier(n_estimators=1000, random_state=0))])

################################################################################
## 04-SVC Classifier
vec = TfidfVectorizer(min_df=3,ngram_range=(1, 2))
svd = TruncatedSVD()
pipe = make_pipeline(vec, svd)
clf = SVC()
trial_04 = make_pipeline(pipe, clf)

#################################################################################
# 05- SGD Classifier
vec=TfidfVectorizer()
clf= SGDClassifier()
trial_05=Pipeline([("vect",vec),("clf",clf)])

#################################################################################



# Training and Prediction by XGBCalssifier

In [ ]:
y_pred=train_predict(trial_02,X_train,y_train,X_test,y_test)

# Evaluation

First we used classification report and confusion matrix for a test set  . Then we show the result of classifcation report and confusion matrix over ten folds to have unibased evaluation metrics. here we have chosen two models among 5 models that we tried.

 ## a) train-test-split

In [ ]:
cr=classification_report(y_test,y_pred)
print(cr)
Color_Confusion_Matrix(y=y,y_test=y_test,y_pred=y_pred)

## b) 10 fold cross valdiation

In [ ]:
cross_validation_accuracy(trial_02,X,y)

In [ ]:
y_pred = cross_val_predict(trial_02, X, y, cv=10)
cr=classification_report(y,y_pred)
print(cr)
cm = confusion_matrix(y_pred,y)
Color_Confusion_Matrix(y=y,y_test=y,y_pred=y_pred)

### ROC curve and AUC value

In [ ]:
AUC = roc_auc_score(y,y_pred)  
fpr, tpr, thresholds = roc_curve(y,y_pred) 
plot_roc_curve(fpr,tpr,AUC)

# Training and Prediction by SGDClassifier

In [ ]:
y_pred=train_predict(trial_05,X_train,y_train,X_test,y_test)

# Evaluation

## a) train-test-split

In [ ]:
cr=classification_report(y_test,y_pred)
print(cr)
Color_Confusion_Matrix(y=y,y_test=y_test,y_pred=y_pred)

## b) 10 fold cross valdiation

In [ ]:
cross_validation_accuracy(trial_05,X,y)

In [ ]:
y_pred = cross_val_predict(trial_05, X, y, cv=10)
cr=classification_report(y,y_pred)
print(cr)
cm = confusion_matrix(y_pred,y)
Color_Confusion_Matrix(y=y,y_test=y,y_pred=y_pred)

### ROC curve and AUC value

In [ ]:
AUC = roc_auc_score(y,y_pred)  
fpr, tpr, thresholds = roc_curve(y,y_pred) 
plot_roc_curve(fpr,tpr,AUC)

# Parameter Tuning on XGB Classifier

In [ ]:
param_grid= {
    'vect__max_df': (0.25, 0.5, 0.75),
    'vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
   'clf__max_iter' : [400],
   'clf__tol' : np.logspace(-4,0,5)
}
trial_02_01 = GridSearchCV(trial_02, param_grid=param_grid, cv=3, n_jobs=-1, verbose=0, error_score=0)

In [ ]:
trial_02_01.fit(X_train,y_train)

In [ ]:
clf_02_01=trial_02_01.best_estimator_

In [ ]:
y_pred=clf_02_01.predict(X_test)

## Evaluation after parameter tuning

In [ ]:
cr=classification_report(y_test,y_pred)
print(cr)
Color_Confusion_Matrix(y=y,y_test=y_test,y_pred=y_pred)

In [ ]:
cross_validation_accuracy(trial_02_01,X,y)

In [ ]:
y_pred = cross_val_predict(trial_02_01, X, y, cv=5)
cr=classification_report(y,y_pred)
print(cr)
cm = confusion_matrix(y_pred,y)
Color_Confusion_Matrix(y=y,y_test=y,y_pred=y_pred)

In [ ]:
AUC = roc_auc_score(y,y_pred)  
fpr, tpr, thresholds = roc_curve(y,y_pred) 
plot_roc_curve(fpr,tpr,AUC)

As you see the results are improved by parameter tuning

# Parameter Tuning on SGD Classifier

In [ ]:
param_grid_01= {
    'clf__alpha': [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e-1, 1e2, 1e3], # learning rate
    'clf__n_iter': [1000], # number of epochs
    'clf__loss': ['log'], 
    'clf__penalty': ['l2'],
    'clf__n_jobs': [-1]
}
trial_05_01 = GridSearchCV(trial_05, param_grid=param_grid_01, cv=5, n_jobs=-1, verbose=0, error_score=0)

In [ ]:
trial_05_01.fit(X_train,y_train)

In [ ]:
clf_05_01=trial_05_01.best_estimator_

In [ ]:
y_pred=train_predict(clf_05_01,X_train,y_train,X_test,y_test)

## Evaluation after parameter tuning

In [ ]:
cr=classification_report(y_test,y_pred)
print(cr)
Color_Confusion_Matrix(y=y,y_test=y_test,y_pred=y_pred)

In [ ]:
cross_validation_accuracy(clf_05_01,X,y)

In [ ]:
y_pred = cross_val_predict(clf_05_01, X, y, cv=5)
cr=classification_report(y,y_pred)
print(cr)
cm = confusion_matrix(y_pred,y)
Color_Confusion_Matrix(y=y,y_test=y,y_pred=y_pred)

In [ ]:
AUC = roc_auc_score(y,y_pred)  
fpr, tpr, thresholds = roc_curve(y,y_pred) 
plot_roc_curve(fpr,tpr,AUC)

# Explaining prediction by XGB classifier using eli5

In [ ]:
from eli5 import show_weights
show_weights(trial_05, top=60)

In [ ]:
import eli5
eli5.show_prediction(clf, X_train[22],vec=vec)

In [ ]:
eli5.show_prediction(clf, X_train[28],vec=vec)

In [ ]:
eli5.show_prediction(clf, X_train[55],vec=vec)

In [ ]:
eli5.show_prediction(clf, X_train[25],vec=vec)

# Subject Object features

ELi5's explanation can give us a motivation to work on subject (also object) features 

In [ ]:
X_f=list(dfExoplanetsNASAannot["s"])



The features here are subjects and objects as following

In [ ]:
X_f[0:5]

In [ ]:
X,y = shuffle(X_f, y, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(
X,y, test_size=0.25, random_state=42)    

# XGB classifier on subject-object features

In [ ]:
clf = XGBClassifier()
vec = DictVectorizer()
pipeline = make_pipeline(vec, clf)



# Evalutaion 

In [ ]:
cross_validation_accuracy(pipeline,X,y)

In [ ]:
y_pred=train_predict(pipeline,X_train,y_train,X_test,y_test)

In [ ]:
cr=classification_report(y_test,y_pred)
print(cr)
Color_Confusion_Matrix(y=y,y_test=y_test,y_pred=y_pred)

In [ ]:
y_pred = cross_val_predict(pipeline, X, y, cv=5)
cr=classification_report(y,y_pred)
print(cr)
cm = confusion_matrix(y_pred,y)
Color_Confusion_Matrix(y=y,y_test=y,y_pred=y_pred)